## ToDos:
- alternate who starts the game... (make it more robus)

In [ ]:
import itertools 
import random
random.seed(10)
from tictactoe import *
import numpy as np
import math
import copy

In [ ]:
# hyper-params
lr = 0.1
discount = 0.8
n_episodes = 1000

EMPTY_FIELD = 0
OUR_AGENT = 1
ENEMY_OPPONENT = 2

In [ ]:
def _flatten(nested):
    flat = []
    for l in nested:
        flat.extend(l)
    else:
        return flat

In [ ]:
def init_lut():
    """
    0 ... empty fields
    1 ... our AGENT's prior moves
    2 ... OPPONENT's moves
    """
    global lut
    lut = {}
    for state in itertools.product([0,1,2],repeat=9): 
        lut[state] = [round(random.uniform(0, 1),1) if s == 0 else float('nan') for s in state]
        
        # check for terminal state
        board = [state[:3],state[3:6],state[6:9]]
        if has_won(board, 1) or has_won(board, 2):
            lut[state] = [0.0 if s == 0 else float('nan') for s in state]

In [ ]:
def select_action(board, get_q=False):
    if get_q:
        return np.nanmax(lut[tuple(_flatten(board))])
    else:
        q_i = int(np.nanargmax(lut[tuple(_flatten(board))]))
        return (math.floor(q_i / 3), q_i % 3)

In [ ]:
def convert_action_to_index(action):
    return action[0]*3 + action[1]

In [ ]:
def update_lut(board, action, value):
    q_i = convert_action_to_index(action)
    lut[tuple(_flatten(board))][q_i] = value

In [ ]:
# initialize the lut
init_lut()

# Train Loop
- maybe show cool grpahic how over time the percentage of who is winning gradually changes

In [ ]:
# run for n of episodes
for e in range(n_episodes):
    
    # one episode
    state_0 = init_board()
    state_1 = copy.deepcopy(state_0)
    action_0 = select_action(state_0)
    action_1 = copy.deepcopy(action_0)
    while not has_won(state_0, 1) and not has_won(state_0, 2) and not is_full(state_0):
        r = 0
        
        # our agent performs move (can lead to reaching final state) (watch out what state we are in here!)
        mark(state_1, 1, *action_0)
        if has_won(state_1, 1):
            r = 3
        elif is_full(state_1):
            r = 2
        else:
            # environment react to it (can ALSO lead to reaching final state)
            row, col = get_ai_move(state_1, 2)
            mark(state_1, 2, row, col)
            if has_won(state_1, 2):
                r = -3
            elif is_full(state_1):
                r = 2
            else:
                # choose A' from S'
                action_1 = select_action(state_1)
            
        # update for ONE particular state ONE action [ Q(S',A') should be 0 if terminal! ]
        if has_won(state_1, 1) or has_won(state_1, 2) or is_full(state_1):
            q_new = select_action(state_0, get_q=True) + lr * (r + discount * 0 - select_action(state_0, get_q=True))
        else:
            q_new = select_action(state_0, get_q=True) + lr * (r + discount * select_action(state_1, get_q=True) - select_action(state_0, get_q=True))

        update_lut(state_0, action_0, q_new)
        
        # prep for next round
        state_0 = copy.deepcopy(state_1)
        action_0 = copy.deepcopy(action_1)
        
        if has_won(state_1, 1) or has_won(state_1, 2) or is_full(state_1):
            break
    # break

## Test Run Through
- afterwards check if maybe you misunderstood something? how did he implement it?

In [ ]:
agent_wins = 0
opponents_wins = 0
draw = 0

for _ in range(n_episodes):
    board = init_board()
    while not has_won(board, 1) and not has_won(board, 2) and not is_full(board):
        agent_move = select_action(board)
        mark(board, 1, *agent_move)
        if not has_won(board, 1) and not is_full(board):
            row, col = get_ai_move(board, 2)
            mark(board, 2, row, col)

    if has_won(board, 1):
        agent_wins += 1
    elif has_won(board, 2):
        opponents_wins += 1
    else:
        draw += 1


print(f'agent_wins: {agent_wins}')
print(f'opponents_wins: {opponents_wins}')
print(f'draw: {draw}')